In [1]:
from scipy.spatial import distance # For calculating Euclidean distance between points
from imutils import face_utils # Utility functions for facial landmarks
import imutils # Utility functions for image processing
import dlib # Library for machine learning and image processing, specifically face detection and landmarks
import cv2 # OpenCV library for image and video processing

In [2]:
def eye_aspect_ratio(eye):
    # Calculate the vertical distances between the landmarks of the eye
	vertical_distance1 = distance.euclidean(eye[1], eye[5])
	vertical_distance2 = distance.euclidean(eye[2], eye[4])
    # Calculate the horizontal distance between the eye landmarks
	horizontal_distance = distance.euclidean(eye[0], eye[3])
    # Calculate the Eye Aspect Ratio (EAR)
	ear = (vertical_distance1 + vertical_distance2) / (2.0 * horizontal_distance)
	return ear

In [3]:
thresh = 0.20 # Threshold value for EAR to detect drowsiness
frame_check = 20 # Number of consecutive frames the EAR should be below the threshold to trigger an alert
# Initialize dlib's face detector (HOG-based) and facial landmarks predictor
detect = dlib.get_frontal_face_detector() 
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
# Extract the indexes for the left and right eye landmarks
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [5]:
# Start capturing video from the webcam
cap=cv2.VideoCapture(0)
ctr=0 # Initialize a counter to track frames where EAR is below the threshold

while cap.isOpened():
    ret, frame=cap.read() # Capture a frame from the webcam
    frame = imutils.resize(frame, width=600) # Resize the frame to a width of 600 pixels
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale
    subjects = detect(gray, 0) # Detect faces in the grayscale frame
    
    for subject in subjects: # Loop over each detected face
        shape = predict(gray, subject) # Predict the facial landmarks
        shape = face_utils.shape_to_np(shape) # Convert the landmarks to a NumPy array
        # Extract the coordinates for the left and right eye
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # Calculate the EAR for both eyes
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # Average the EAR for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        
        # If the EAR is below the threshold, increment the counter
        if ear < thresh:
            ctr += 1
            # If the counter exceeds the frame_check threshold, display the alert
            if ctr >= frame_check:
                cv2.putText(frame, "Wake up!!", (150, 250),	cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        else:
            ctr = 0 # Reset the counter if EAR is above the threshold

    # Display the resulting frame with the alert, if triggered
    cv2.imshow("Frame", frame)
    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows() # Close all OpenCV windows
        cap.release() # Release the webcam
        break